In [1]:
import pandas as pd
import time
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression

import numpy as np
from matplotlib import pyplot as plt
import warnings
from skopt import BayesSearchCV, space
from sklearn.metrics import average_precision_score
# use feature importance for feature selection
from numpy import sort
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
warnings.filterwarnings("ignore")
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
### Parameters
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularPredictor
from autogluon.common import space
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.model_selection import KFold
#import ts_fc_functions as ff
import import_ipynb
from darts import TimeSeries
from darts.models import NBEATSModel, XGBModel, TiDEModel#, LightGBMModel
from darts.models import NHiTSModel,LightGBMModel,RNNModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape, r2_score, mae, rmse
import torch

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


In [ ]:
def data_processing_training_scale(data,value_to_predict,resample_rate,min_ts_length,imputation,max_train_length,prediction_length,padding,scale_series):

    resampled_df = resample_ts(data,resample_rate,min_ts_length,imputation,max_train_length,value_to_predict)

    resampled_df, scaler = scale_ts(resampled_df,value_to_predict)
    valid_series_dict_full, test_series_dict_full, train_series_dict_full = ts_dictionary(resampled_df,prediction_length,value_to_predict,resample_rate)

        # Pad the series
    if padding == True:
        train_series_dict_full = pad_series_dict(train_series_dict_full, max_train_length, pad_value=0)
        valid_series_dict_full = pad_series_dict(valid_series_dict_full, max_train_length, pad_value=0)   

    return train_series_dict_full,valid_series_dict_full,resampled_df, scaler

def data_processing_training(data,value_to_predict,resample_rate,min_ts_length,imputation,max_train_length,prediction_length,padding):

    resampled_df = resample_ts(data,resample_rate,min_ts_length,imputation,max_train_length,value_to_predict)
    valid_series_dict_full, test_series_dict_full, train_series_dict_full = ts_dictionary(resampled_df,prediction_length,value_to_predict,resample_rate)
    if padding == True:
        train_series_dict_full = pad_series_dict(train_series_dict_full, max_train_length, pad_value=0)
        valid_series_dict_full = pad_series_dict(valid_series_dict_full, max_train_length, pad_value=0)   

    return train_series_dict_full,valid_series_dict_full,resampled_df

        


In [ ]:
def process_use_case_3_data(data_use_case_3, start_date, end_date):
    df_sorted = data_use_case_3.sort_values(by='performedDateTime').reset_index(drop=True)
    df_no_duplicates = df_sorted.drop_duplicates(subset=['performedDateTime', 'subject_reference', 'encounter_reference']).reset_index(drop=True)

    # Generate a date range with unique dates
    date_range = pd.date_range(start=start_date, end=end_date, freq="D")

    # Duplicate each date to have two entries per day
    date_range = date_range.repeat(2)
    df_no_duplicates["date"] = pd.to_datetime(df_no_duplicates["date"])


    ctscans = pd.DataFrame(date_range, columns=["date"])
    ctscans["time_of_day"] = ctscans.groupby("date").cumcount().map({0: "a_morning", 1: "b_evening"})

    count_df = df_no_duplicates.groupby(["date", "time_of_day"]).size().reset_index(name="count_scans")
    ctscans = pd.merge(ctscans, count_df, on=["date", "time_of_day"], how="left")
    ctscans["count_scans"] = ctscans["count_scans"].fillna(0).astype(int)
    ctscans = ctscans.drop(index=0)

    # Reset the index if desired, to avoid a missing index number
    ctscans = ctscans.reset_index(drop=True)

    # Create a new column 'shift_count' initialized to NaN
    ctscans["shift_count"] = np.nan

    # Iterate over the rows and sum count_scans for each consecutive "b_evening" and "a_morning" pair
    for i in range(len(ctscans) - 1):
        if ctscans.loc[i, "time_of_day"] == "b_evening" and ctscans.loc[i + 1, "time_of_day"] == "a_morning":
            # Sum count_scans for the consecutive "b_evening" and "a_morning"
            shift_sum = ctscans.loc[i, "count_scans"] + ctscans.loc[i + 1, "count_scans"]
            
            # Assign the result to the first row of the pair ("b_evening" row)
            ctscans.loc[i, "shift_count"] = shift_sum

    # Optionally, forward-fill or set NaN values to 0 in 'shift_count'
    ctscans["shift_count"] = ctscans["shift_count"].fillna(0).astype(int)
    ctscans = ctscans[ctscans["time_of_day"] != "a_morning"].reset_index(drop=True)
    ctscans["day_of_week"] = ctscans["date"].dt.day_name()
    last_friday = ctscans[ctscans["day_of_week"] == "Friday"]["date"].max()

    # Step 2: Initialize ts_id counter and end_date for the first 4-week period
    current_id = 1
    end_date = last_friday

    # Step 3: Create an empty 'ts_id' column
    ctscans["ts_id"] = 0

    # Step 4: Assign ts_id to each 4-week (28-day) period, going backwards from the last Friday
    while end_date >= ctscans["date"].min():
        start_date = end_date - pd.Timedelta(weeks=4)

        # Assign the current ts_id to dates within the 4-week range
        ctscans.loc[(ctscans["date"] > start_date) & (ctscans["date"] <= end_date), "ts_id"] = current_id

        # Move to the next 4-week period back in time
        end_date = start_date
        current_id += 1
    ctscans
    return ctscans

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os


def bootstrap_metrics_roc(
    actual_values,
    predicted_values,
    group_ids,
    n_bootstrap=1000,
    ci_percentile=95,
    threshold_value=8,
    comparison_operator="<"
):


    # Helper: comparison logic
    def apply_threshold(x, operator, threshold):
        if operator == "<":
            return x < threshold
        elif operator == "<=":
            return x <= threshold
        elif operator == ">":
            return x > threshold
        elif operator == ">=":
            return x >= threshold
        elif operator == "==":
            return x == threshold
        else:
            raise ValueError("Invalid comparison_operator")

    # Combine data
    data = pd.DataFrame({
        "actual": actual_values,
        "predicted": predicted_values,
        "group_id": group_ids
    })

    # Aggregate at group level
    grouped = data.groupby("group_id").agg(
        actual_condition=("actual", lambda x: apply_threshold(x, comparison_operator, threshold_value).any()),
        predicted_condition=("predicted", lambda x: apply_threshold(x, comparison_operator, threshold_value).any()),
        predicted_mean=("predicted", "mean")
    )

    # Exact confusion matrix
    tp_exact = ((grouped["actual_condition"]) & (grouped["predicted_condition"])).sum()
    fn_exact = ((grouped["actual_condition"]) & (~grouped["predicted_condition"])).sum()
    fp_exact = ((~grouped["actual_condition"]) & (grouped["predicted_condition"])).sum()
    tn_exact = ((~grouped["actual_condition"]) & (~grouped["predicted_condition"])).sum()

    # Exact classification metrics
    sensitivity_exact = tp_exact / (tp_exact + fn_exact) if (tp_exact + fn_exact) > 0 else 0
    specificity_exact = tn_exact / (tn_exact + fp_exact) if (tn_exact + fp_exact) > 0 else 0
    ppv_exact = tp_exact / (tp_exact + fp_exact) if (tp_exact + fp_exact) > 0 else 0
    npv_exact = tn_exact / (tn_exact + fn_exact) if (tn_exact + fn_exact) > 0 else 0
    accuracy_exact = (tp_exact + tn_exact) / (tp_exact + fn_exact + fp_exact + tn_exact) if (tp_exact + fn_exact + fp_exact + tn_exact) > 0 else 0
    f1_exact = (2 * tp_exact) / (2 * tp_exact + fp_exact + fn_exact) if (2 * tp_exact + fp_exact + fn_exact) > 0 else 0
    mcc_exact = ((tp_exact * tn_exact) - (fp_exact * fn_exact)) / (
        np.sqrt((tp_exact + fp_exact) * (tp_exact + fn_exact) * (tn_exact + fp_exact) * (tn_exact + fn_exact))
        if (tp_exact + fp_exact) > 0 and (tp_exact + fn_exact) > 0 and (tn_exact + fp_exact) > 0 and (tn_exact + fn_exact) > 0
        else 1
    )

    # Regression metrics 
    mae_exact = np.mean(np.abs(actual_values - predicted_values))
    mse_exact = np.mean((actual_values - predicted_values) ** 2)
    rmse_exact = np.sqrt(mse_exact)
    mape_exact = np.mean(np.abs((actual_values - predicted_values) / np.where(actual_values == 0, np.nan, actual_values))) * 100
    smape_exact = np.mean(2 * np.abs(actual_values - predicted_values) / (np.abs(actual_values) + np.abs(predicted_values))) * 100

    #  ROC and AUC (continuous predicted_mean) 
    try:
        auc_exact = roc_auc_score(grouped["actual_condition"], grouped["predicted_mean"])
        fpr, tpr, _ = roc_curve(grouped["actual_condition"], grouped["predicted_mean"])
    except ValueError:
        auc_exact = np.nan
        fpr, tpr = [0, 1], [0, 1]

    #  Plot ROC 
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_exact:.2f})")
    plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC) Curve")
    plt.legend(loc="lower right")
    plt.grid(True)
    roc_curve_path = "roc_curve.png"
    plt.savefig(roc_curve_path)
    plt.close()

    # Bootstrap containers 
    auc_bootstrap = []
    tp_b, fn_b, fp_b, tn_b = [], [], [], []
    sens_b, spec_b, ppv_b, npv_b, acc_b, f1_b, mcc_b = [], [], [], [], [], [], []
    mae_b, mse_b, rmse_b, mape_b, smape_b = [], [], [], [], []

    #  Bootstrapping 
    for _ in range(n_bootstrap):
        sampled_ids = np.random.choice(grouped.index, size=len(grouped), replace=True)
        resampled_data = data[data["group_id"].isin(sampled_ids)]
        resampled_grouped = resampled_data.groupby("group_id").agg(
            actual_condition=("actual", lambda x: apply_threshold(x, comparison_operator, threshold_value).any()),
            predicted_condition=("predicted", lambda x: apply_threshold(x, comparison_operator, threshold_value).any()),
            predicted_mean=("predicted", "mean")
        )

        # Confusion matrix
        tp = ((resampled_grouped["actual_condition"]) & (resampled_grouped["predicted_condition"])).sum()
        fn = ((resampled_grouped["actual_condition"]) & (~resampled_grouped["predicted_condition"])).sum()
        fp = ((~resampled_grouped["actual_condition"]) & (resampled_grouped["predicted_condition"])).sum()
        tn = ((~resampled_grouped["actual_condition"]) & (~resampled_grouped["predicted_condition"])).sum()

        tp_b.append(tp); fn_b.append(fn); fp_b.append(fp); tn_b.append(tn)

        # Metrics
        sens_b.append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        spec_b.append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        ppv_b.append(tp / (tp + fp) if (tp + fp) > 0 else 0)
        npv_b.append(tn / (tn + fn) if (tn + fn) > 0 else 0)
        acc_b.append((tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0)
        f1_b.append((2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0)
        mcc_b.append(((tp * tn) - (fp * fn)) / (
            np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
            if (tp + fp) > 0 and (tp + fn) > 0 and (tn + fp) > 0 and (tn + fn) > 0
            else 1
        ))

        try:
            auc_b = roc_auc_score(resampled_grouped["actual_condition"], resampled_grouped["predicted_mean"])
        except ValueError:
            auc_b = np.nan
        auc_bootstrap.append(auc_b)

        # Regression metrics
        a_r, p_r = resampled_data["actual"].values, resampled_data["predicted"].values
        mae_b.append(np.mean(np.abs(a_r - p_r)))
        mse_b.append(np.mean((a_r - p_r) ** 2))
        rmse_b.append(np.sqrt(mse_b[-1]))
        mape_b.append(np.mean(np.abs((a_r - p_r) / np.where(a_r == 0, np.nan, a_r))) * 100)
        smape_b.append(np.mean(2 * np.abs(a_r - p_r) / (np.abs(a_r) + np.abs(p_r))) * 100)

    # --- Helper: CI ---
    def ci(v):
        return np.nanpercentile(v, (100 - ci_percentile) / 2), np.nanpercentile(v, 100 - (100 - ci_percentile) / 2)

    # Classification metrics DF
    classification_metrics = pd.DataFrame({
        "Metric": ["Sensitivity", "Specificity", "PPV", "NPV", "Accuracy", "F1", "MCC", "AUC", "TP", "FN", "FP", "TN"],
        "Point Estimate": [sensitivity_exact, specificity_exact, ppv_exact, npv_exact, accuracy_exact, f1_exact, mcc_exact, auc_exact, tp_exact, fn_exact, fp_exact, tn_exact],
        "CI Lower": [ci(sens_b)[0], ci(spec_b)[0], ci(ppv_b)[0], ci(npv_b)[0], ci(acc_b)[0], ci(f1_b)[0], ci(mcc_b)[0], ci(auc_bootstrap)[0], ci(tp_b)[0], ci(fn_b)[0], ci(fp_b)[0], ci(tn_b)[0]],
        "CI Upper": [ci(sens_b)[1], ci(spec_b)[1], ci(ppv_b)[1], ci(npv_b)[1], ci(acc_b)[1], ci(f1_b)[1], ci(mcc_b)[1], ci(auc_bootstrap)[1], ci(tp_b)[1], ci(fn_b)[1], ci(fp_b)[1], ci(tn_b)[1]],
    })

    regression_metrics = pd.DataFrame({
        "Metric": ["MAE", "MSE", "RMSE", "MAPE", "SMAPE"],
        "Point Estimate": [mae_exact, mse_exact, rmse_exact, mape_exact, smape_exact],
        "CI Lower": [ci(mae_b)[0], ci(mse_b)[0], ci(rmse_b)[0], ci(mape_b)[0], ci(smape_b)[0]],
        "CI Upper": [ci(mae_b)[1], ci(mse_b)[1], ci(rmse_b)[1], ci(mape_b)[1], ci(smape_b)[1]],
    })

    return classification_metrics, regression_metrics, roc_curve_path


In [ ]:
def timesfm_data_ci_roc_(
    model_name,
    output_dir_forecasting,
    output_dir_classification,
    forecasting_filename,
    classification_filename,
    resampled_df,
    tfm,
    scaler,
    prediction_length,
    value_to_predict,
    threshold_value=8,
    comparison_operator='<',
    n_bootstrap=1000
):

    # Split into train/test
    full_data_test = resampled_df.groupby("encounter_id").tail(prediction_length)
    full_data_train = resampled_df.drop(full_data_test.index)
    full_data_train_ = full_data_train.rename(columns={'recorded_time': 'ds', 'encounter_id': 'unique_id','value':'y'})
    full_data_test_ = full_data_test.rename(columns={'recorded_time': 'ds', 'encounter_id': 'unique_id','value':'y'})

    # Forecast
    forecasts = tfm.forecast_on_df(
        inputs=full_data_train_,
        freq="D",
        value_name="y",
        num_jobs=3,
    )

    # Helper: inverse transform
    def rescale_to_original(scaled_df, scaler, columns_to_rescale):
        scaled_df = scaled_df.copy()
        scaled_df[columns_to_rescale] = scaler.inverse_transform(scaled_df[columns_to_rescale])
        return scaled_df

    columns_to_rescale_actual = ["y"]
    columns_to_rescale_predict = ["timesfm"]
    actuals = rescale_to_original(full_data_test_, scaler, columns_to_rescale_actual)
    predictions = rescale_to_original(forecasts, scaler, columns_to_rescale_predict)

    # Bootstrap metrics for forecasting
    def bootstrap_timellm_metrics_length(forecasts_df, actual_values_df, prediction_length, n_bootstrap, ci_percentile=95):
        mae_bootstrap, mse_bootstrap, rmse_bootstrap, smape_bootstrap, mape_bootstrap = [], [], [], [], []

        merged_df = pd.merge(forecasts_df.reset_index(), actual_values_df, on=['unique_id', 'ds'])
        unique_ids = merged_df['unique_id'].unique()

        for _ in range(n_bootstrap):
            sampled_ids = np.random.choice(unique_ids, size=len(unique_ids), replace=True)
            sampled_data = merged_df[merged_df['unique_id'].isin(sampled_ids)]
            if len(sampled_ids) == 0:
                continue

            total_mae = total_mse = total_smape = total_mape = 0

            for unique_id in sampled_ids:
                data = sampled_data[sampled_data['unique_id'] == unique_id].sort_values(by='ds')
                if data.empty:
                    continue

                forecast_values = data['timesfm'].values[-prediction_length:]
                true_values = data['y'].values[-prediction_length:]
                if len(forecast_values) == 0 or len(true_values) == 0:
                    continue

                abs_err = np.abs(forecast_values - true_values)
                mae = abs_err.mean()
                mse = (abs_err ** 2).mean()
                denominator = (np.abs(true_values) + np.abs(forecast_values)) / 2
                smape = (abs_err / denominator).mean() * 100
                mape = np.mean(np.abs((true_values - forecast_values) / np.where(true_values == 0, np.nan, true_values))) * 100

                total_mae += mae
                total_mse += mse
                total_smape += smape
                total_mape += mape

            mae_bootstrap.append(total_mae / len(sampled_ids))
            mse_bootstrap.append(total_mse / len(sampled_ids))
            rmse_bootstrap.append(np.sqrt(total_mse / len(sampled_ids)))
            smape_bootstrap.append(total_smape / len(sampled_ids))
            mape_bootstrap.append(total_mape / len(sampled_ids))

        metrics = {
            "mae": (np.mean(mae_bootstrap), np.percentile(mae_bootstrap, 2.5), np.percentile(mae_bootstrap, 97.5)),
            "mse": (np.mean(mse_bootstrap), np.percentile(mse_bootstrap, 2.5), np.percentile(mse_bootstrap, 97.5)),
            "rmse": (np.mean(rmse_bootstrap), np.percentile(rmse_bootstrap, 2.5), np.percentile(rmse_bootstrap, 97.5)),
            "smape": (np.mean(smape_bootstrap), np.percentile(smape_bootstrap, 2.5), np.percentile(smape_bootstrap, 97.5)),
            "mape": (np.mean(mape_bootstrap), np.percentile(mape_bootstrap, 2.5), np.percentile(mape_bootstrap, 97.5)),
        }
        return metrics, merged_df

    metrics, merged_data = bootstrap_timellm_metrics_length(predictions, actuals, prediction_length, n_bootstrap, 95)

    # Save forecasting metrics
    forecasting_metrics_df = pd.DataFrame(metrics)
    os.makedirs(output_dir_forecasting, exist_ok=True)
    forecasting_csv_path = os.path.join(output_dir_forecasting, forecasting_filename)
    forecasting_metrics_df.to_csv(forecasting_csv_path, index=False)
    print(f"Forecasting metrics saved to {forecasting_csv_path}")

    merged_data = merged_data.rename(columns={'y': 'actual', 'timesfm': 'predictions'})

    # Apply condition threshold 
    def apply_threshold(x, operator, threshold):
        if operator == '<':
            return x < threshold
        elif operator == '<=':
            return x <= threshold
        elif operator == '>':
            return x > threshold
        elif operator == '>=':
            return x >= threshold
        elif operator == '==':
            return x == threshold
        else:
            raise ValueError("Invalid comparison_operator. Choose from '<', '<=', '>', '>=', '=='.")

    merged_data['actual_condition'] = merged_data.groupby('unique_id')['actual'].transform(lambda x: apply_threshold(x, comparison_operator, threshold_value).any())
    merged_data['predicted_condition'] = merged_data.groupby('unique_id')['predictions'].transform(lambda x: apply_threshold(x, comparison_operator, threshold_value).any())

    merged_data = merged_data.reset_index(drop=True)
    unique_encounters = merged_data.drop_duplicates(subset=['unique_id'])

    # Confusion matrix
    confusion_conditions = [
        (unique_encounters['actual_condition'] == True) & (unique_encounters['predicted_condition'] == True),
        (unique_encounters['actual_condition'] == True) & (unique_encounters['predicted_condition'] == False),
        (unique_encounters['actual_condition'] == False) & (unique_encounters['predicted_condition'] == True),
        (unique_encounters['actual_condition'] == False) & (unique_encounters['predicted_condition'] == False),
    ]
    confusion_choices = ['TP', 'FN', 'FP', 'TN']
    unique_encounters['confusion'] = np.select(confusion_conditions, confusion_choices, default='Unknown')

    # Bootstrap classification metrics
    confusion_matrix_bootstrap = {'TP': [], 'FN': [], 'FP': [], 'TN': []}
    metrics_bootstrap = {'sensitivity': [], 'specificity': [], 'ppv': [], 'npv': [], 'accuracy': [], 'f1': [], 'mcc': []}
    auc_scores = []

    for _ in range(n_bootstrap):
        bootstrap_sample = unique_encounters.sample(frac=1, replace=True)
        tp = ((bootstrap_sample['actual_condition']) & (bootstrap_sample['predicted_condition'])).sum()
        fn = ((bootstrap_sample['actual_condition']) & (~bootstrap_sample['predicted_condition'])).sum()
        fp = ((~bootstrap_sample['actual_condition']) & (bootstrap_sample['predicted_condition'])).sum()
        tn = ((~bootstrap_sample['actual_condition']) & (~bootstrap_sample['predicted_condition'])).sum()

        confusion_matrix_bootstrap['TP'].append(tp)
        confusion_matrix_bootstrap['FN'].append(fn)
        confusion_matrix_bootstrap['FP'].append(fp)
        confusion_matrix_bootstrap['TN'].append(tn)

        metrics_bootstrap['sensitivity'].append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        metrics_bootstrap['specificity'].append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        metrics_bootstrap['ppv'].append(tp / (tp + fp) if (tp + fp) > 0 else 0)
        metrics_bootstrap['npv'].append(tn / (tn + fn) if (tn + fn) > 0 else 0)
        metrics_bootstrap['accuracy'].append((tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0)
        metrics_bootstrap['f1'].append((2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0)
        metrics_bootstrap['mcc'].append(((tp * tn) - (fp * fn)) / (
            ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        ) if ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) > 0 else 0)

        try:
            auc = roc_auc_score(bootstrap_sample['actual_condition'], bootstrap_sample['predictions'])
        except ValueError:
            auc = np.nan
        auc_scores.append(auc)

    # Compute CI
    def ci_dict(values):
        return {
            "Mean": np.nanmean(values),
            "CI Lower": np.nanpercentile(values, 2.5),
            "CI Upper": np.nanpercentile(values, 97.5),
        }

    confusion_matrix_ci = {k: ci_dict(v) for k, v in confusion_matrix_bootstrap.items()}
    metrics_ci = {k: ci_dict(v) for k, v in metrics_bootstrap.items()}
    auc_ci = ci_dict(auc_scores)

    # Prepare classification metrics
    classification_metrics_data = [
        {"Metric": metric.capitalize(), "Point Estimate": stats["Mean"], "95% CI Lower": stats["CI Lower"], "95% CI Upper": stats["CI Upper"]}
        for metric, stats in metrics_ci.items()
    ]
    for key, stats in confusion_matrix_ci.items():
        classification_metrics_data.append({
            "Metric": f"Avg {key}",
            "Point Estimate": stats["Mean"],
            "95% CI Lower": stats["CI Lower"],
            "95% CI Upper": stats["CI Upper"]
        })
    classification_metrics_data.append({
        "Metric": "AUC",
        "Point Estimate": auc_ci["Mean"],
        "95% CI Lower": auc_ci["CI Lower"],
        "95% CI Upper": auc_ci["CI Upper"],
    })

    classification_metrics_df = pd.DataFrame(classification_metrics_data)
    os.makedirs(output_dir_classification, exist_ok=True)
    classification_csv_path = os.path.join(output_dir_classification, classification_filename)
    classification_metrics_df.to_csv(classification_csv_path, index=False)
    print(f"Classification metrics saved to {classification_csv_path}")

    # Plot ROC
    def plot_roc_curve(actuals, predictions, output_path):
        if len(np.unique(actuals)) < 2:
            print("ROC curve not plotted: only one class present.")
            return
        fpr, tpr, _ = roc_curve(actuals, predictions)
        auc = roc_auc_score(actuals, predictions)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc:.2f})')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.savefig(output_path)
        plt.close()
        print(f"ROC curve saved to {output_path}")

    roc_curve_path = os.path.join(output_dir_classification, 'roc_curve.png')
    plot_roc_curve(unique_encounters['actual_condition'], unique_encounters['predictions'], roc_curve_path)

    return predictions, actuals, merged_data, unique_encounters['actual_condition'], unique_encounters['predictions']

def predict_proba_ci_roc_general(
    output_dir_classification,
    classification_filename,
    output_dir_forecasting,
    forecasting_filename,
    model,
    train_series_dict,
    valid_series_dict,
    prediction_length,
    scaler,
    n_bootstrap=1000,
    threshold_value=8,
    comparison_operator="<"
):


    # Generate predictions
    pred = model.predict(n=prediction_length, series=list(train_series_dict.values()), num_samples=500)

    # Collect actuals and predictions 
    actual_values, predicted_values, keys = [], [], []

    for key, ts in valid_series_dict.items():
        actual = ts.values()[-prediction_length:].flatten()
        predicted = np.median(pred[key].all_values(), axis=1)
        actual_values.append(actual)
        predicted_values.append(predicted)
        keys.append(key)

    #  Convert and rescale 
    actual_values_np = scaler.inverse_transform(np.array(actual_values))
    predicted_values_np = scaler.inverse_transform(np.array(predicted_values))

    #  Build full long-form data 
    all_records = []
    for i, key in enumerate(keys):
        for t in range(prediction_length):
            all_records.append({
                "group_id": key,
                "actual": actual_values_np[i, t],
                "predicted": predicted_values_np[i, t]
            })
    df_all = pd.DataFrame(all_records)

    #  Compute metrics 
    classification_metrics, regression_metrics, roc_path = bootstrap_metrics_roc(
        df_all["actual"].values,
        df_all["predicted"].values,
        df_all["group_id"].values,
        n_bootstrap=n_bootstrap,
        threshold_value=threshold_value,
        comparison_operator=comparison_operator
    )

    #  Save outputs 
    os.makedirs(output_dir_classification, exist_ok=True)
    classification_metrics.to_csv(os.path.join(output_dir_classification, classification_filename), index=False)

    os.makedirs(output_dir_forecasting, exist_ok=True)
    regression_metrics.to_csv(os.path.join(output_dir_forecasting, forecasting_filename), index=False)

    print(f"Classification metrics saved to {classification_filename}")
    print(f"Forecasting metrics saved to {forecasting_filename}")
    print(f"ROC curve saved to {roc_path}")

    return predicted_values, actual_values, keys, df_all, classification_metrics, regression_metrics




In [ ]:


def process_use_case_3_data_with_scaling(data_use_case_3,start_date,end_date):
    df_sorted = data_use_case_3.sort_values(by='performedDateTime').reset_index(drop=True)
    df_no_duplicates = df_sorted.drop_duplicates(subset=['performedDateTime', 'subject_reference', 'encounter_reference']).reset_index(drop=True)

    # Generate a date range with unique dates
    date_range = pd.date_range(start=start_date, end=end_date, freq="D")

    # Duplicate each date to have two entries per day
    date_range = date_range.repeat(2)
    df_no_duplicates["date"] = pd.to_datetime(df_no_duplicates["date"])

    ctscans = pd.DataFrame(date_range, columns=["date"])
    ctscans["time_of_day"] = ctscans.groupby("date").cumcount().map({0: "a_morning", 1: "b_evening"})

    count_df = df_no_duplicates.groupby(["date", "time_of_day"]).size().reset_index(name="count_scans")
    ctscans = pd.merge(ctscans, count_df, on=["date", "time_of_day"], how="left")
    ctscans["count_scans"] = ctscans["count_scans"].fillna(0).astype(int)
    ctscans = ctscans.drop(index=0)

    # Reset the index if desired, to avoid a missing index number
    ctscans = ctscans.reset_index(drop=True)

    # Create a new column 'shift_count' initialized to NaN
    ctscans["shift_count"] = np.nan

    # Iterate over the rows and sum count_scans for each consecutive "b_evening" and "a_morning" pair
    for i in range(len(ctscans) - 1):
        if ctscans.loc[i, "time_of_day"] == "b_evening" and ctscans.loc[i + 1, "time_of_day"] == "a_morning":
            # Sum count_scans for the consecutive "b_evening" and "a_morning"
            shift_sum = ctscans.loc[i, "count_scans"] + ctscans.loc[i + 1, "count_scans"]
            
            # Assign the result to the first row of the pair ("b_evening" row)
            ctscans.loc[i, "shift_count"] = shift_sum

    # Optionally, forward-fill or set NaN values to 0 in 'shift_count'
    ctscans["shift_count"] = ctscans["shift_count"].fillna(0).astype(int)
    ctscans = ctscans[ctscans["time_of_day"] != "a_morning"].reset_index(drop=True)
    ctscans["day_of_week"] = ctscans["date"].dt.day_name()
    last_friday = ctscans[ctscans["day_of_week"] == "Friday"]["date"].max()

    # Step 2: Initialize ts_id counter and end_date for the first 4-week period
    current_id = 1
    end_date = last_friday

    # Step 3: Create an empty 'ts_id' column
    ctscans["ts_id"] = 0

    # Step 4: Assign ts_id to each 4-week (28-day) period, going backwards from the last Friday
    while end_date >= ctscans["date"].min():
        start_date = end_date - pd.Timedelta(weeks=4)

        # Assign the current ts_id to dates within the 4-week range
        ctscans.loc[(ctscans["date"] > start_date) & (ctscans["date"] <= end_date), "ts_id"] = current_id

        # Move to the next 4-week period back in time
        end_date = start_date
        current_id += 1

    # Min-Max scaling for 'count_scans' and 'shift_count'
    scaler = MinMaxScaler()
    scaled_columns = ["shift_count"]

    # Apply Min-Max Scaling
    ctscans[scaled_columns] = scaler.fit_transform(ctscans[scaled_columns])

    return ctscans, scaler

In [ ]:
def create_rolling_windows_with_weekday_prediction(data, target_col, lookback, horizon, prediction_days):
    """
    Creates rolling windows for a time series while preserving dates for each window.
    Args:
        data (DataFrame): DataFrame containing the time series.
        target_col (str): Column name of the target variable.
        lookback (int): Number of steps to look back.
        horizon (int): Number of steps to forecast.
        prediction_days (list): List of valid day names for the prediction window (e.g., ['Monday', ..., 'Friday']).
    Returns:
        X (list): List of dictionaries containing input sequences and their corresponding dates.
        y (list): List of dictionaries containing target sequences and their corresponding dates.
        dates (list): Start dates of each rolling window.
    """
    X, y, dates = [], [], []

    for i in range(len(data) - lookback - horizon + 1):
        # Define the input (lookback) window
        window = data.iloc[i:i + lookback]
        input_values = window[target_col].values
        input_dates = window['date'].values
        
        # Define the prediction (horizon) window
        prediction_window = data.iloc[i + lookback:i + lookback + horizon]
        target_values = prediction_window[prediction_window['day_of_week'].isin(prediction_days)][target_col].values
        target_dates = prediction_window[prediction_window['day_of_week'].isin(prediction_days)]['date'].values

        # Ensure the prediction window matches the expected length
        if len(target_values) == len(prediction_days):
            X.append({"values": input_values, "dates": input_dates})
            y.append({"values": target_values, "dates": target_dates})
            dates.append(data.iloc[i + lookback]['date'])

    def convert_to_dataframe_with_id(data, id_name):
        """
        Converts a list of dictionaries with 'values' and 'dates' into a DataFrame, 
        assigning a unique id to each sequence.
        Args:
            data (list): List of dictionaries with keys 'values' and 'dates'.
            id_name (str): Column name for the unique identifier.
        Returns:
            DataFrame: Combined DataFrame of all sequences with dates, values, and ids.
        """
        df_list = []
        for idx, entry in enumerate(data, start=1):
            temp_df = pd.DataFrame({
                "date": entry["dates"],
                "value": entry["values"],
                id_name: idx  # Assign a unique ID for each sequence
            })
            df_list.append(temp_df)
        return pd.concat(df_list, ignore_index=True)

    # Example X and y conversion with IDs

    X_df = convert_to_dataframe_with_id(X, id_name="X_id")
    y_df = convert_to_dataframe_with_id(y, id_name="y_id")
    def merge_and_order_dataframes(X_df, y_df, X_id_col="X_id", y_id_col="y_id"):
        """
        Merges X_df and y_df, ordering by id and date, and distinguishing X and y records.
        Args:
            X_df (DataFrame): DataFrame for X with date, value, and ID.
            y_df (DataFrame): DataFrame for y with date, value, and ID.
            X_id_col (str): Column name for X IDs.
            y_id_col (str): Column name for y IDs.
        Returns:
            DataFrame: Merged and ordered DataFrame.
        """
        # Add a label column to distinguish X and y
        X_df["type"] = "X"
        y_df["type"] = "y"
        
        # Rename ID columns to align for merging
        X_df = X_df.rename(columns={X_id_col: "id"})
        y_df = y_df.rename(columns={y_id_col: "id"})

        # Concatenate X and y DataFrames
        merged_df = pd.concat([X_df, y_df], ignore_index=True)
        
        # Order by id and date
        merged_df = merged_df.sort_values(by=["id", "date"]).reset_index(drop=True)
        
        return merged_df,X_df,y_df
    merged_df,X_df,y_df = merge_and_order_dataframes(X_df, y_df)

    return X, y, dates,X_df,y_df,merged_df

In [ ]:
def ts_dictionary(full_data, X_df, y_df):
    value_to_predict = 'value'
    
    def create_timeseries_dict(grouped_data):
        series_dict = {}
        for encounter_id, group in grouped_data:
            group['date'] = pd.to_datetime(group['date'])  # Ensure 'date' is datetime
            group = group.sort_values(by='date')
            group = group.set_index('date')  # Set 'date' as index
            ts_series = TimeSeries.from_dataframe(group, value_cols=value_to_predict, freq='D')
            series_dict[encounter_id] = ts_series
        return series_dict

    grouped_train = X_df.groupby('id')
    grouped_test = y_df.groupby('id')
    grouped_validation = full_data.groupby('id')

    train_series_dict = create_timeseries_dict(grouped_train)
    test_series_dict = create_timeseries_dict(grouped_test)
    valid_series_dict = create_timeseries_dict(grouped_validation)

    return valid_series_dict, test_series_dict, train_series_dict



In [ ]:
def prepare_train_validation_split(train_series_dict, valid_series_dict, input_chunk_length, output_chunk_length):
    """
    Adjusts train_series_dict to ensure no overlap with targets in valid_series_dict.
    Ensures strict separation to prevent information leakage.
    
    Parameters:
    - train_series_dict: Dictionary of training series (time series without targets).
    - valid_series_dict: Dictionary of validation series (includes prediction targets).
    - input_chunk_length: Length of the input window for the model.
    - output_chunk_length: Length of the prediction horizon.
    
    Returns:
    - adjusted_train_series_dict: Training series dictionary without overlapping target data.
    - valid_series_dict: Validation series dictionary remains unchanged.
    """
    # Initialize the adjusted training series dictionary
    adjusted_train_series_dict = {}
    for key, train_series in train_series_dict.items():
        #print('TRAIN:', key)
        # Check if the key exists in both dictionaries
        if key not in valid_series_dict:
            continue

        # Extract the corresponding validation series
        valid_series = valid_series_dict[key]

        # Identify the last index of the training data
        train_end_idx = len(train_series) - output_chunk_length  # Leave gap for prediction horizon
        #print("train_end_idx", train_end_idx)
        # Truncate training series to avoid overlap with validation targets
        adjusted_train_series = train_series[:train_end_idx]
        adjusted_train_series_dict[key] = adjusted_train_series
    #print('adjusted_train_series_dict:', train_series_dict)
    return train_series_dict, valid_series_dict

    #return train_series_dict, valid_series_dict

In [ ]:
def custom_time_series_split(train_series_dict, valid_series_dict, n_splits):
    """
    Custom time-based splitting function for dictionaries of time series.
    
    Parameters:
    - train_series_dict: Dictionary of training time series.
    - valid_series_dict: Dictionary of validation time series.
    - n_splits: Number of splits for cross-validation.
    
    Returns:
    - A list of (train_keys, valid_keys) pairs for each fold.
    """
    keys = list(train_series_dict.keys())
    #print(keys)
    kf = KFold(n_splits=n_splits, shuffle=False)

    splits = []
    for train_idx, valid_idx in kf.split(keys):
        train_keys = [keys[i] for i in train_idx]
        valid_keys = [keys[i] for i in valid_idx]
        splits.append((train_keys, valid_keys))
    
    return splits

In [ ]:
def bootstrap_metrics_auto(actual_values, predicted_values, n_bootstrap=1000, ci_percentile=95):
    # Initialize arrays to store metrics across bootstraps
    mae_bootstrap = []
    mse_bootstrap = []
    rmse_bootstrap = []
    mape_bootstrap = []
    smape_bootstrap = []

    n_samples = len(actual_values)

    for _ in range(n_bootstrap):
        # Sample with replacement
        resample_indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        actual_resample = np.array([actual_values[i] for i in resample_indices])
        predicted_resample = np.array([predicted_values[i] for i in resample_indices])

        # Calculate metrics for this bootstrap sample
        mae = np.mean(np.abs(actual_resample - predicted_resample))
        mse = np.mean((actual_resample - predicted_resample) ** 2)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((actual_resample - predicted_resample) / actual_resample)) * 100
        smape = np.mean(np.abs(actual_resample - predicted_resample) / (np.abs(actual_resample) + np.abs(predicted_resample))) * 100

        # Append the results
        mae_bootstrap.append(mae)
        mse_bootstrap.append(mse)
        rmse_bootstrap.append(rmse)
        mape_bootstrap.append(mape)
        smape_bootstrap.append(smape)

    # Calculate mean and confidence intervals
    metrics = {
        "mae": (np.mean(mae_bootstrap), np.percentile(mae_bootstrap, (100 - ci_percentile) / 2), np.percentile(mae_bootstrap, 100 - (100 - ci_percentile) / 2)),
        "mse": (np.mean(mse_bootstrap), np.percentile(mse_bootstrap, (100 - ci_percentile) / 2), np.percentile(mse_bootstrap, 100 - (100 - ci_percentile) / 2)),
        "rmse": (np.mean(rmse_bootstrap), np.percentile(rmse_bootstrap, (100 - ci_percentile) / 2), np.percentile(rmse_bootstrap, 100 - (100 - ci_percentile) / 2)),
        "mape": (np.mean(mape_bootstrap), np.percentile(mape_bootstrap, (100 - ci_percentile) / 2), np.percentile(mape_bootstrap, 100 - (100 - ci_percentile) / 2)),
        "smape": (np.mean(smape_bootstrap), np.percentile(smape_bootstrap, (100 - ci_percentile) / 2), np.percentile(smape_bootstrap, 100 - (100 - ci_percentile) / 2)),
    }

    return metrics

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from autogluon.timeseries import TimeSeriesDataFrame


def autogluon_data_ci_roc_general(
    model_name,
    output_dir_forecasting,
    output_dir_classification,
    forecasting_filename,
    classification_filename,
    resampled_df,
    predictor,
    scaler,
    prediction_length,
    value_to_predict,
    threshold_value=8,
    comparison_operator="<",
    n_bootstrap=1000
):

    # Prepare data
    resampled_df = resampled_df.rename(columns={"id": "encounter_id", "date": "recorded_time"})

    # Split into train/test
    def split_train_test(group):
        test_rows = group.nlargest(prediction_length, "recorded_time")
        train_rows = group.drop(test_rows.index)
        return train_rows, test_rows

    train_list, test_list = [], []
    for _, group in resampled_df.groupby("encounter_id"):
        train_rows, test_rows = split_train_test(group)
        train_list.append(train_rows)
        test_list.append(test_rows)

    train_data = pd.concat(train_list).sort_values(by=["encounter_id", "recorded_time"]).reset_index(drop=True)
    test_data = pd.concat(test_list).sort_values(by=["encounter_id", "recorded_time"]).reset_index(drop=True)

    train_data_ = train_data[["encounter_id", "recorded_time", value_to_predict]].copy()
    test_data_ = test_data[["encounter_id", "recorded_time", value_to_predict]].copy()
    train_data_["recorded_time"] = train_data_["recorded_time"].dt.tz_localize(None)
    test_data_["recorded_time"] = test_data_["recorded_time"].dt.tz_localize(None)

    train_ts = TimeSeriesDataFrame.from_data_frame(train_data_, id_column="encounter_id", timestamp_column="recorded_time")
    test_ts = TimeSeriesDataFrame.from_data_frame(test_data_, id_column="encounter_id", timestamp_column="recorded_time")

    # Predict with AutoGluon
    predictions = predictor.predict(train_ts, model=model_name)
    predictions_ = predictions[["mean", "0.05", "0.95"]]

    # Merge with actuals
    merged_data = test_ts.join(predictions_, how="inner")

    actuals = merged_data[value_to_predict].values.reshape(-1, 1)
    preds = merged_data["mean"].values.reshape(-1, 1)

    # Rescale
    def rescale_predictions(scaled_values, scaler):
        if len(scaled_values.shape) == 1:
            scaled_values = scaled_values.reshape(-1, 1)
        return scaler.inverse_transform(scaled_values).flatten()

    actuals = rescale_predictions(actuals, scaler)
    preds = rescale_predictions(preds, scaler)

    # --- Forecasting metrics with bootstrapping ---
    def bootstrap_metrics(actual, predicted, n_bootstrap=1000, ci=95):
        mae_list, mse_list, rmse_list, smape_list, mape_list = [], [], [], [], []
        for _ in range(n_bootstrap):
            idx = np.random.choice(len(actual), len(actual), replace=True)
            a, p = actual[idx], predicted[idx]
            abs_err = np.abs(p - a)
            mae = abs_err.mean()
            mse = np.mean((p - a) ** 2)
            rmse = np.sqrt(mse)
            denom = (np.abs(a) + np.abs(p)) / 2
            smape = np.mean(abs_err / denom) * 100
            mape = np.mean(np.abs((a - p) / np.where(a == 0, np.nan, a))) * 100
            mae_list.append(mae)
            mse_list.append(mse)
            rmse_list.append(rmse)
            smape_list.append(smape)
            mape_list.append(mape)

        def ci_vals(v):
            return (np.mean(v), np.percentile(v, 2.5), np.percentile(v, 97.5))

        return {
            "MAE": ci_vals(mae_list),
            "MSE": ci_vals(mse_list),
            "RMSE": ci_vals(rmse_list),
            "SMAPE": ci_vals(smape_list),
            "MAPE": ci_vals(mape_list),
        }

    metrics_forecasting = bootstrap_metrics(actuals, preds, n_bootstrap=n_bootstrap)

    # Save forecasting metrics
    forecasting_metrics_df = pd.DataFrame(
        [
            {"Metric": m, "Point Estimate": v[0], "95% CI Lower": v[1], "95% CI Upper": v[2]}
            for m, v in metrics_forecasting.items()
        ]
    )
    os.makedirs(output_dir_forecasting, exist_ok=True)
    forecasting_path = os.path.join(output_dir_forecasting, forecasting_filename)
    forecasting_metrics_df.to_csv(forecasting_path, index=False)
    print(f"Forecasting metrics saved to {forecasting_path}")

    # --- Condition detection ---
    merged_data["actual"] = actuals.flatten()
    merged_data["predictions"] = preds.flatten()

    def apply_threshold(x, operator, threshold):
        if operator == "<":
            return x < threshold
        elif operator == "<=":
            return x <= threshold
        elif operator == ">":
            return x > threshold
        elif operator == ">=":
            return x >= threshold
        elif operator == "==":
            return x == threshold
        else:
            raise ValueError("Invalid comparison_operator. Choose from '<', '<=', '>', '>=', '=='.")

    merged_data["actual_condition"] = merged_data.groupby("encounter_id")["actual"].transform(
        lambda x: apply_threshold(x, comparison_operator, threshold_value).any()
    )
    merged_data["predicted_condition"] = merged_data.groupby("encounter_id")["predictions"].transform(
        lambda x: apply_threshold(x, comparison_operator, threshold_value).any()
    )

    unique_encounters = merged_data.drop_duplicates(subset=["encounter_id"])

    # Confusion matrix setup
    confusion_conditions = [
        (unique_encounters["actual_condition"]) & (unique_encounters["predicted_condition"]),
        (unique_encounters["actual_condition"]) & (~unique_encounters["predicted_condition"]),
        (~unique_encounters["actual_condition"]) & (unique_encounters["predicted_condition"]),
        (~unique_encounters["actual_condition"]) & (~unique_encounters["predicted_condition"]),
    ]
    confusion_choices = ["TP", "FN", "FP", "TN"]
    unique_encounters["confusion"] = np.select(confusion_conditions, confusion_choices, default="Unknown")

    # --- Bootstrapping classification metrics ---
    confusion_matrix_bootstrap = {"TP": [], "FN": [], "FP": [], "TN": []}
    metrics_bootstrap = {"sensitivity": [], "specificity": [], "ppv": [], "npv": [], "accuracy": [], "f1": [], "mcc": []}
    auc_scores = []

    for _ in range(n_bootstrap):
        bs = unique_encounters.sample(frac=1, replace=True)
        tp = ((bs["actual_condition"]) & (bs["predicted_condition"])).sum()
        fn = ((bs["actual_condition"]) & (~bs["predicted_condition"])).sum()
        fp = ((~bs["actual_condition"]) & (bs["predicted_condition"])).sum()
        tn = ((~bs["actual_condition"]) & (~bs["predicted_condition"])).sum()

        confusion_matrix_bootstrap["TP"].append(tp)
        confusion_matrix_bootstrap["FN"].append(fn)
        confusion_matrix_bootstrap["FP"].append(fp)
        confusion_matrix_bootstrap["TN"].append(tn)

        metrics_bootstrap["sensitivity"].append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        metrics_bootstrap["specificity"].append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        metrics_bootstrap["ppv"].append(tp / (tp + fp) if (tp + fp) > 0 else 0)
        metrics_bootstrap["npv"].append(tn / (tn + fn) if (tn + fn) > 0 else 0)
        metrics_bootstrap["accuracy"].append((tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0)
        metrics_bootstrap["f1"].append((2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0)
        metrics_bootstrap["mcc"].append(((tp * tn) - (fp * fn)) / (
            ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        ) if ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) > 0 else 0)

        try:
            auc = roc_auc_score(bs["actual_condition"], bs["predictions"])
        except ValueError:
            auc = np.nan
        auc_scores.append(auc)

    def ci_dict(values):
        return {
            "Mean": np.nanmean(values),
            "CI Lower": np.nanpercentile(values, 2.5),
            "CI Upper": np.nanpercentile(values, 97.5),
        }

    confusion_matrix_ci = {k: ci_dict(v) for k, v in confusion_matrix_bootstrap.items()}
    metrics_ci = {k: ci_dict(v) for k, v in metrics_bootstrap.items()}
    auc_ci = ci_dict(auc_scores)

    classification_metrics_data = [
        {"Metric": metric.capitalize(), "Point Estimate": stats["Mean"], "95% CI Lower": stats["CI Lower"], "95% CI Upper": stats["CI Upper"]}
        for metric, stats in metrics_ci.items()
    ]
    for key, stats in confusion_matrix_ci.items():
        classification_metrics_data.append({
            "Metric": f"Avg {key}",
            "Point Estimate": stats["Mean"],
            "95% CI Lower": stats["CI Lower"],
            "95% CI Upper": stats["CI Upper"]
        })
    classification_metrics_data.append({
        "Metric": "AUC",
        "Point Estimate": auc_ci["Mean"],
        "95% CI Lower": auc_ci["CI Lower"],
        "95% CI Upper": auc_ci["CI Upper"]
    })

    classification_metrics_df = pd.DataFrame(classification_metrics_data)
    os.makedirs(output_dir_classification, exist_ok=True)
    class_path = os.path.join(output_dir_classification, classification_filename)
    classification_metrics_df.to_csv(class_path, index=False)
    print(f"Classification metrics saved to {class_path}")

    # --- ROC Plot ---
    def plot_roc_curve(actuals, predictions, output_path):
        if len(np.unique(actuals)) < 2:
            print("ROC curve not plotted: only one class present.")
            return
        fpr, tpr, _ = roc_curve(actuals, predictions)
        auc = roc_auc_score(actuals, predictions)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc:.2f})")
        plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("Receiver Operating Characteristic (ROC) Curve")
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.savefig(output_path)
        plt.close()
        print(f"ROC curve saved to {output_path}")

    roc_path = os.path.join(output_dir_classification, "roc_curve.png")
    plot_roc_curve(unique_encounters["actual_condition"], unique_encounters["predictions"], roc_path)

    return preds, actuals, train_ts, test_ts, merged_data, unique_encounters["actual_condition"], unique_encounters["predictions"]


In [ ]:
def timellm_data_ci_roc_(
    model_name,
    output_dir_forecasting,
    output_dir_classification,
    forecasting_filename,
    classification_filename,
    resampled_df,
    nf,
    scaler,
    prediction_length,
    value_to_predict,
    threshold_value=8,
    comparison_operator='<',
    n_bootstrap=1000
):

    # Split into train/test
    full_data_test = resampled_df.groupby("encounter_id").tail(prediction_length)
    full_data_train = resampled_df.drop(full_data_test.index)
    full_data_train_ = full_data_train.rename(columns={'recorded_time': 'ds', 'encounter_id': 'unique_id','value':'y'})
    full_data_test_ = full_data_test.rename(columns={'recorded_time': 'ds', 'encounter_id': 'unique_id','value':'y'})

    # Forecast with TimeLLM model
    forecasts = nf.predict(full_data_train_)

    # Helper: inverse transform
    def rescale_to_original(scaled_df, scaler, columns_to_rescale):
        scaled_df = scaled_df.copy()
        scaled_df[columns_to_rescale] = scaler.inverse_transform(scaled_df[columns_to_rescale])
        return scaled_df

    columns_to_rescale_actual = ["y"]
    columns_to_rescale_predict = ["TimeLLM"]
    actuals = rescale_to_original(full_data_test_, scaler, columns_to_rescale_actual)
    predictions = rescale_to_original(forecasts, scaler, columns_to_rescale_predict)

    # --- Bootstrap metrics for forecasting ---
    def bootstrap_timellm_metrics_length(forecasts_df, actual_values_df, prediction_length, n_bootstrap, ci_percentile=95):
        mae_bootstrap, mse_bootstrap, rmse_bootstrap, smape_bootstrap, mape_bootstrap = [], [], [], [], []

        merged_df = pd.merge(forecasts_df.reset_index(), actual_values_df, on=['unique_id', 'ds'])
        unique_ids = merged_df['unique_id'].unique()

        for _ in range(n_bootstrap):
            sampled_ids = np.random.choice(unique_ids, size=len(unique_ids), replace=True)
            sampled_data = merged_df[merged_df['unique_id'].isin(sampled_ids)]
            if len(sampled_ids) == 0:
                continue

            total_mae = total_mse = total_smape = total_mape = 0

            for unique_id in sampled_ids:
                data = sampled_data[sampled_data['unique_id'] == unique_id].sort_values(by='ds')
                if data.empty:
                    continue

                forecast_values = data['TimeLLM'].values[-prediction_length:]
                true_values = data['y'].values[-prediction_length:]
                if len(forecast_values) == 0 or len(true_values) == 0:
                    continue

                abs_err = np.abs(forecast_values - true_values)
                mae = abs_err.mean()
                mse = (abs_err ** 2).mean()
                denominator = (np.abs(true_values) + np.abs(forecast_values)) / 2
                smape = (abs_err / denominator).mean() * 100
                mape = np.mean(np.abs((true_values - forecast_values) / np.where(true_values == 0, np.nan, true_values))) * 100

                total_mae += mae
                total_mse += mse
                total_smape += smape
                total_mape += mape

            mae_bootstrap.append(total_mae / len(sampled_ids))
            mse_bootstrap.append(total_mse / len(sampled_ids))
            rmse_bootstrap.append(np.sqrt(total_mse / len(sampled_ids)))
            smape_bootstrap.append(total_smape / len(sampled_ids))
            mape_bootstrap.append(total_mape / len(sampled_ids))

        metrics = {
            "mae": (np.mean(mae_bootstrap), np.percentile(mae_bootstrap, 2.5), np.percentile(mae_bootstrap, 97.5)),
            "mse": (np.mean(mse_bootstrap), np.percentile(mse_bootstrap, 2.5), np.percentile(mse_bootstrap, 97.5)),
            "rmse": (np.mean(rmse_bootstrap), np.percentile(rmse_bootstrap, 2.5), np.percentile(rmse_bootstrap, 97.5)),
            "smape": (np.mean(smape_bootstrap), np.percentile(smape_bootstrap, 2.5), np.percentile(smape_bootstrap, 97.5)),
            "mape": (np.mean(mape_bootstrap), np.percentile(mape_bootstrap, 2.5), np.percentile(mape_bootstrap, 97.5)),
        }
        return metrics, merged_df

    metrics, merged_data = bootstrap_timellm_metrics_length(predictions, actuals, prediction_length, n_bootstrap, 95)

    # Save forecasting metrics
    forecasting_metrics_df = pd.DataFrame(metrics)
    os.makedirs(output_dir_forecasting, exist_ok=True)
    forecasting_csv_path = os.path.join(output_dir_forecasting, forecasting_filename)
    forecasting_metrics_df.to_csv(forecasting_csv_path, index=False)
    print(f"Forecasting metrics saved to {forecasting_csv_path}")

    merged_data = merged_data.rename(columns={'y': 'actual', 'TimeLLM': 'predictions'})

    # --- Apply condition threshold ---
    def apply_threshold(x, operator, threshold):
        if operator == '<':
            return x < threshold
        elif operator == '<=':
            return x <= threshold
        elif operator == '>':
            return x > threshold
        elif operator == '>=':
            return x >= threshold
        elif operator == '==':
            return x == threshold
        else:
            raise ValueError("Invalid comparison_operator. Choose from '<', '<=', '>', '>=', '=='.")

    merged_data['actual_condition'] = merged_data.groupby('unique_id')['actual'].transform(lambda x: apply_threshold(x, comparison_operator, threshold_value).any())
    merged_data['predicted_condition'] = merged_data.groupby('unique_id')['predictions'].transform(lambda x: apply_threshold(x, comparison_operator, threshold_value).any())

    merged_data = merged_data.reset_index(drop=True)
    unique_encounters = merged_data.drop_duplicates(subset=['unique_id'])

    # Confusion matrix
    confusion_conditions = [
        (unique_encounters['actual_condition']) & (unique_encounters['predicted_condition']),
        (unique_encounters['actual_condition']) & (~unique_encounters['predicted_condition']),
        (~unique_encounters['actual_condition']) & (unique_encounters['predicted_condition']),
        (~unique_encounters['actual_condition']) & (~unique_encounters['predicted_condition']),
    ]
    confusion_choices = ['TP', 'FN', 'FP', 'TN']
    unique_encounters['confusion'] = np.select(confusion_conditions, confusion_choices, default='Unknown')

    # Bootstrap classification metrics
    confusion_matrix_bootstrap = {'TP': [], 'FN': [], 'FP': [], 'TN': []}
    metrics_bootstrap = {'sensitivity': [], 'specificity': [], 'ppv': [], 'npv': [], 'accuracy': [], 'f1': [], 'mcc': []}
    auc_scores = []

    for _ in range(n_bootstrap):
        bootstrap_sample = unique_encounters.sample(frac=1, replace=True)
        tp = ((bootstrap_sample['actual_condition']) & (bootstrap_sample['predicted_condition'])).sum()
        fn = ((bootstrap_sample['actual_condition']) & (~bootstrap_sample['predicted_condition'])).sum()
        fp = ((~bootstrap_sample['actual_condition']) & (bootstrap_sample['predicted_condition'])).sum()
        tn = ((~bootstrap_sample['actual_condition']) & (~bootstrap_sample['predicted_condition'])).sum()

        confusion_matrix_bootstrap['TP'].append(tp)
        confusion_matrix_bootstrap['FN'].append(fn)
        confusion_matrix_bootstrap['FP'].append(fp)
        confusion_matrix_bootstrap['TN'].append(tn)

        metrics_bootstrap['sensitivity'].append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        metrics_bootstrap['specificity'].append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        metrics_bootstrap['ppv'].append(tp / (tp + fp) if (tp + fp) > 0 else 0)
        metrics_bootstrap['npv'].append(tn / (tn + fn) if (tn + fn) > 0 else 0)
        metrics_bootstrap['accuracy'].append((tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0)
        metrics_bootstrap['f1'].append((2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0)
        metrics_bootstrap['mcc'].append(((tp * tn) - (fp * fn)) / (
            ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        ) if ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) > 0 else 0)

        try:
            auc = roc_auc_score(bootstrap_sample['actual_condition'], bootstrap_sample['predictions'])
        except ValueError:
            auc = np.nan
        auc_scores.append(auc)

    # Compute CI
    def ci_dict(values):
        return {
            "Mean": np.nanmean(values),
            "CI Lower": np.nanpercentile(values, 2.5),
            "CI Upper": np.nanpercentile(values, 97.5),
        }

    confusion_matrix_ci = {k: ci_dict(v) for k, v in confusion_matrix_bootstrap.items()}
    metrics_ci = {k: ci_dict(v) for k, v in metrics_bootstrap.items()}
    auc_ci = ci_dict(auc_scores)

    # Prepare classification metrics
    classification_metrics_data = [
        {"Metric": metric.capitalize(), "Point Estimate": stats["Mean"], "95% CI Lower": stats["CI Lower"], "95% CI Upper": stats["CI Upper"]}
        for metric, stats in metrics_ci.items()
    ]
    for key, stats in confusion_matrix_ci.items():
        classification_metrics_data.append({
            "Metric": f"Avg {key}",
            "Point Estimate": stats["Mean"],
            "95% CI Lower": stats["CI Lower"],
            "95% CI Upper": stats["CI Upper"]
        })
    classification_metrics_data.append({
        "Metric": "AUC",
        "Point Estimate": auc_ci["Mean"],
        "95% CI Lower": auc_ci["CI Lower"],
        "95% CI Upper": auc_ci["CI Upper"],
    })

    classification_metrics_df = pd.DataFrame(classification_metrics_data)
    os.makedirs(output_dir_classification, exist_ok=True)
    classification_csv_path = os.path.join(output_dir_classification, classification_filename)
    classification_metrics_df.to_csv(classification_csv_path, index=False)
    print(f"Classification metrics saved to {classification_csv_path}")

    # Plot ROC
    def plot_roc_curve(actuals, predictions, output_path):
        if len(np.unique(actuals)) < 2:
            print("ROC curve not plotted: only one class present.")
            return
        fpr, tpr, _ = roc_curve(actuals, predictions)
        auc = roc_auc_score(actuals, predictions)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc:.2f})')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.savefig(output_path)
        plt.close()
        print(f"ROC curve saved to {output_path}")

    roc_curve_path = os.path.join(output_dir_classification, 'roc_curve.png')
    plot_roc_curve(unique_encounters['actual_condition'], unique_encounters['predictions'], roc_curve_path)

    return predictions, actuals, merged_data, unique_encounters['actual_condition'], unique_encounters['predictions']
